In [32]:
import json
import re
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

In [33]:
from transformers import AutoModelForCausalLM
AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: bb6651a2-c18d-4be7-aacc-cd36a962f638)')' thrown while requesting HEAD https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0/resolve/main/config.json
Retrying in 1s [Retry 1/5].


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rot

In [34]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float16
)

device = "mps" if torch.backends.mps.is_available() else "cpu"
model.to(device)
model.eval()


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rot

In [35]:
with open('basic prompt.txt') as f:
    basic_prompt = f.read()

with open('enhanced prompt.txt') as f:
    enhanced_prompt = f.read()

with open('constrained prompt.txt') as f:
    constrained_prompt = f.read()

prompt_dict = {
    "basic": basic_prompt,
    "enhanced": enhanced_prompt,
    "constrained": constrained_prompt}



In [36]:
validation_trials = pd.read_csv('validation_trials.csv')

# IMPORTANT: start small
validation_trials = validation_trials.head(1)

print("Columns in dataset:")
print(validation_trials.columns)


Columns in dataset:
Index(['nctId', 'briefTitle', 'conditions', 'specialty', 'clean_text'], dtype='object')


In [37]:
def extract_json(text):
    try:
        match = re.search(r"\{.*\}", text, re.DOTALL)
        if match:
            return json.loads(match.group())
        return None
    except:
        return None


In [38]:
def generate_response(prompt):

    messages = [{"role": "user", "content": prompt}]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(text, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=400,
            temperature=0.2
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [39]:
results = []

for prompt_name, prompt_template in prompt_dict.items():

    for _, trial in tqdm(validation_trials.iterrows(), total=len(validation_trials)):

        trial_text = str(trial['conditions'])

        full_prompt = prompt_template.format(trial_text=trial_text)

        output_text = generate_response(full_prompt)
        extraction_json = extract_json(output_text)

        status = "success" if extraction_json else "json_error"

        results.append({
            "prompt": prompt_name,
            "nct_id": trial.get("nct_id", "unknown"),
            "extraction": extraction_json,
            "status": status,
            "raw_output": output_text
        })


  0%|          | 0/1 [00:00<?, ?it/s]


KeyError: '\n    "inclusion_criteria"'

In [40]:
with open("prompt_test_results.json", "w") as f:
    json.dump(results, f, indent=2)

print("\nExtraction complete. Results saved.")


Extraction complete. Results saved.
